In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split

train_dir = "..\isec-ic\\dataset\\train"
validation_dir = "..\isec-ic\\dataset\\valid"
test_dir = "..\isec-ic\\dataset\\test"

current_dir = os.path.dirname(os.path.realpath(__file__ if '__file__' in locals() else os.getcwd()))
train_dir = os.path.join(current_dir, train_dir)
validation_dir = os.path.join(current_dir, validation_dir)
test_dir = os.path.join(current_dir, test_dir)

categories = ["bacterialspot", "healthy","lateblight","leafmold", "mosaicvirus","yellowleafcurlvirus" ,"spidermite","septorialeafspot" ]

def load_images_from_folder(folder):
    images = []
    labels = []
    subfolders = os.listdir(folder)
    for subfolder in subfolders:
        category = subfolder.split("_")[-1].lower()
        if category in categories:
            print("Loading images from category: " + category + " and subfolder: " + subfolder)
            subfolder_path = os.path.join(folder, subfolder)
            for filename in os.listdir(subfolder_path):
                img = Image.open(os.path.join(subfolder_path, filename))
                img = img.resize((64, 64))
                img = np.array(img) / 255.0
                images.append(img)
                labels.append(categories.index(category))
        else:
            print("Category: " + category + " from subfolder " + subfolder + " is not in the list of categories.")
    return np.array(images), np.array(labels)


x_train, y_train = load_images_from_folder(train_dir)
x_validation, y_validation = load_images_from_folder(validation_dir)
x_test, y_test = load_images_from_folder(test_dir)

print("Images loaded successfully")

reduction_ratio = 0.8

x_validation, x_reduction_validation, y_validation, y_reduction_validation = train_test_split(x_validation, y_validation, test_size=reduction_ratio, random_state=42)
print("Conjunto de validação reduzido em 80%")

x_train, x_reduction, y_train, y_reduction = train_test_split(x_train, y_train, test_size=reduction_ratio, random_state=42)
print("Conjunto de treino reduzido em 80%")

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout

def create_cnn_model(dropout_rate, learning_rate):
    #base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
    #base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(64, 64, 3)))
    base_model = Xception(weights='imagenet', include_top=False, input_tensor=Input(shape=(64, 64, 3)))

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)  # Flatten the output layer to 1 dimension
    x = Dense(512, activation='relu')(x)   #  First dense layer
    x = Dropout(dropout_rate)(x)  # Dropout to reduce overfitting
    predictions = Dense(len(categories), activation='softmax')(x)  # Output layer
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

print("Create_cnn_model function loaded")

In [ ]:
import os
import psutil

# Define a função para limitar a afinidade do processador
def limitar_afinidade(processo, num_cores):
    if num_cores < 1:
        num_cores = 1
    elif num_cores > psutil.cpu_count(logical=False):
        num_cores = psutil.cpu_count(logical=False)

    processo.cpu_affinity(list(range(num_cores)))

# Obtém o ID do processo atual
pid = os.getpid()

# Define o número de núcleos desejado (metade dos núcleos disponíveis)
metade_dos_cores = psutil.cpu_count(logical=False) // 2

# Limita a afinidade do processador
limitar_afinidade(psutil.Process(pid), metade_dos_cores)

# Restaura a afinidade do processador para todos os núcleos após a execução do código
limitar_afinidade(psutil.Process(pid), psutil.cpu_count(logical=False))


if tf.test.is_gpu_available():
    print("GPU disponível. Usando GPU para treinamento.")
else:
    print("GPU não disponível. Usando CPU para treinamento.")

In [ ]:
import numpy as np
import random
from sklearn.model_selection import KFold
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import roc_auc_score

#dropout_rates = np.linspace(0.1, 1.0, 5)  # 5 valores igualmente espaçados entre 0.1 e 1.0
#learning_rates = np.logspace(-4, -2, 5)   # 5 valores em escala logarítmica entre 0.0001 e 0.01

# Especificando os valores diretamente
#dropout_rates = [0.1, 0.325, 0.55, 0.775, 1.0]  # Valores específicos para dropout_rates
#learning_rates = [0.0001, 0.00031623, 0.001, 0.00316228, 0.01]  # Valores específicos para learning_rates

dropout_rates = [0.1, 0.325]  # Valores específicos para dropout_rates
learning_rates = [0.0001]  # Valores específicos para learning_rates
k_n_splits=2

def evaluate_model_cv(dropout_rate, learning_rate, x_train, y_train, n_splits=5):
    print(f" - Evaluating with cross validation, splits={n_splits}")
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    auc_scores = []

    for train_index, val_index in kfold.split(x_train):
        model = create_cnn_model(dropout_rate, learning_rate)
        model.fit(x_train[train_index], y_train[train_index], epochs=5, batch_size=32, verbose=1)
        
        y_pred = model.predict(x_train[val_index])
        y_val_categorical = to_categorical(y_train[val_index], num_classes=len(categories))
        auc_score = roc_auc_score(y_val_categorical, y_pred, multi_class='ovr')
        auc_scores.append(auc_score)

    return np.mean(auc_scores)


def grid_search(x_train, y_train, dropout_rates, learning_rates, n_splits=5):
    best_score = -np.inf
    best_params = {'dropout_rate': None, 'learning_rate': None}
    best_auc_score = None

    for dropout_rate in dropout_rates:
        for learning_rate in learning_rates:
            print(f"Testing params: dropout_rate: {dropout_rate}, learning_rate: {learning_rate}")
            auc_score = evaluate_model_cv(dropout_rate, learning_rate, x_train, y_train, n_splits)

            if auc_score > best_score:
                best_score = auc_score
                best_params = {'dropout_rate': dropout_rate, 'learning_rate': learning_rate}
                best_auc_score = auc_score
                print(f"New best score: AUC={auc_score}, with parameters: dropout_rate={dropout_rate}, learning_rate={learning_rate}")

    return best_params, best_auc_score


best_params, best_auc_score = grid_search(x_train, y_train, dropout_rates, learning_rates, n_splits=k_n_splits)
print(f"Melhores parâmetros encontrados: {best_params}, Melhor pontuação AUC: {best_auc_score}")



In [ ]:
print("Avaliando o modelo no conjunto de teste...")
loss_test, test_accuracy = best_model.evaluate(x_test, y_test, verbose=0)

print(f"Acurácia no Teste: {test_accuracy}, Loss: {loss_test}")